---
>「見えるぞ！私にも敵が見える！」\
>シャア・アズナブル
---

# 物体検出


## 物体検出手法

画像の中の複数の物体を認識し、その場所をBounding Boxと呼ばれる四角で示す
- さらにセグメンテーションという手法もあり、こちらは塗り分ける

物体検出手法には著名な手法が多く、シンプルにまとめると次のようになる

- Yolo
  - 一連の物体検出手法YOLO（You Only Look Once）の1つで、v2と比較して特に小さいものの検出精度が改善された
  - Yolo V1からYolo V3まで同一作者により設計され、その後は異なる作者によりYolo V4およびYolo V5が提案されている
    - 従って、Yolo V3までがYoloという研究者もいる
  - 傾向として、FPS(認識フレームレート)が速く、その割に精度が高いことを特徴としている
  - 小さい解像度から大きい解像度に情報を伝搬して物体を検出する構造を有する

- SSD
  - YOLOと並び称されることが多いが、SSDではそれぞれの解像度から直接物体を検出する構造を有する
  - 構造的にはSSDもYOLO V3も大きな違いはなく、SSDは情報を一方向のみにしか伝えていないが、YoloV3はGlobal情報をLocal情報の組み合わせを実現している

論文記載のそれぞれのネットワーク構造は次のとおりである

<img src="http://class.west.sd.keio.ac.jp/dataai/text/ssd-yolo.png" width=500>

ここでは、YoloV3を扱い、実際に実装する


# YoLoの設計

## YoLoの原理

画像ごと異なる対象物体の大きさの違いに対応するため、小スケール(Scale1)$13 \times 13$、中スケール$26 \times 26$、大スケール$52 \times 52$の3つの対象の異なる3次元テンソルを得るように学習する
- 例えば、図のScale 1に相当する$13 \times 13$のスケールでは、$13 \times 13$であることから入力画像を$13\times 13$のグリッドで分割する
- さらに各グリッドで3つのBoxを予測するため、これをBox1, Box2, Box3とする
- 各Boxで得られる(得るべき)情報は、Bounding Boxやラベルなどの情報である
  - Bouding boxは、$(tx, ty, tw, th)$で要素数4
  - 検出のObjectness Score(検出対称であるかどうかを表すフラグ)で要素数1
  - 80クラス分類でワンホットのため要素数80
- Box一つで$3\times(4+1+80)$つまり255次元の情報が獲得され、これが$13 \times 13$個存在するため、結果的に$13 \times 13 \times 255$となる
- 他も同様である

<img src="http://class.west.sd.keio.ac.jp/dataai/text/yololearn.png" width=800>

上図において、仮にGround TruthのBouding Boxの中心があるグリッドセルの中、例えば、鳥の画像の赤いBoxにあったばあい、該当する物体のBoxの予測はこの赤いグリッドセルが担当する
- つまり、同じグリッドセルは3つの物質以上を担当できないという制限がある
- Ground Truth とは学習やテストに使用する実データのこと

赤いセルは鳥のBoxを予測することになり、Objectness Scoreが1に、ほかのセルは0になる
- 各セルは予めサイズの異なる3つのPrior Box(S, M, L)が割り当てられており、Ground Truthと一番大きさの近いPrior Boxを選択する
  - 実際は、Ground truthであるBounding Boxと重なった面積が一番大きいPrior Boxを選択する
- 更に座標オフセットの調整も行う

## Yoloのモデル

### 全体の構造
まず、YoLoV3のモデル構造を示す

<img src="http://class.west.sd.keio.ac.jp/dataai/text/yolov3.png" witdh=600>

このように、主に75層の畳み込み層から構成されており、先に述べた物体の特徴量を抽出する
- Fully connected層を利用していないため、入力画像のサイズを任意に設定できる
- Pooling Layerを用いず、Strideが2の畳み込み層を用いることでダウンサンプリングを行う
  - スケール不変な特徴量を次の層までへ伝播できる
- ResNetとFPN構造を利用して検出精度を向上している
  - 図のResidual BlockがResNet構造に相当し、Shortcut Pathを有する構造である
  - 層が深くなると学習が困難となるが、Shortcut Pathによりある層における最適な出力を学習するのではなく、前層の入力を参照した残差関数を学習するようになり、特徴量の学習をしやすくなる
  - これにより、複雑な特徴量H(x)が古い特徴量xに新しく学習した残差F(x)を足し合わせる形で構成される(階差数列と似た概念)

<img src="http://class.west.sd.keio.ac.jp/dataai/text/yoloresnet.png" width=500>

### 異なるスケールへの対応
特徴マップはネットワークの深さとともに縮小するため、小さい物体の検出もそれに呼応して難易度が上がる
- 直感的に大きさによって異なる段階で物体を検出するとよいと考えるのがSSD

しかしながら、各特徴マップにおける特徴量は深さによって変化しているため、そう簡単ではない
- 最初は低レベルなエッジ、カラー、物体の大まかな位置などの特徴量を学習しているが、層の深くなるにつれてハイレベルな物体情報である犬、猫、車といった特徴量の学習に変化するため、検出にはどうしてもハイレベルな特徴量が必要となる
- これがSSDの弱点

そこで、YOLOv3では、スケール対策としてFeature Pylamid Network(FPN)構造を利用する
- 全体ネットワーク図において、Conv Blockの$13\times 13$から、$26 \times 26$にConcatenateするパスや、さらにそれを$52 \times 52$にConcatenateするパスが存在するなど、大から小のパスと、小から大の双方向のパスが存在する

### Softmaxを利用しない

多値分類では、ワンホット化しSoftmaxを利用する場合が殆どである
- しかしながら、分類数が多くなると例えばwomanとpersonのようにラベルが重なることがあり、完全にワンホットにすることが困難となる場合も想定される
- そこで、logistic関数を利用している

# YoLoの実装

## データの準備

今回は、COCO128を利用する
- データセットとしてはそれほど大きくない

In [1]:
import os
if not os.path.exists('COCO128.zip'):
  #!wget "https://drive.google.com/uc?export=download&id=141bVQDo-x421CMN0o9vS3SkIY75kbi54" -O COCO128.zip
  !wget https://keio.box.com/shared/static/64r487vugf8l385r1lflmp35g0zd63nv -O COCO128.zip
  !unzip COCO128.zip

--2024-08-22 16:08:36--  https://keio.box.com/shared/static/64r487vugf8l385r1lflmp35g0zd63nv
Resolving keio.box.com (keio.box.com)... 74.112.186.157
Connecting to keio.box.com (keio.box.com)|74.112.186.157|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/64r487vugf8l385r1lflmp35g0zd63nv [following]
--2024-08-22 16:08:36--  https://keio.box.com/public/static/64r487vugf8l385r1lflmp35g0zd63nv
Reusing existing connection to keio.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://keio.app.box.com/public/static/64r487vugf8l385r1lflmp35g0zd63nv [following]
--2024-08-22 16:08:36--  https://keio.app.box.com/public/static/64r487vugf8l385r1lflmp35g0zd63nv
Resolving keio.app.box.com (keio.app.box.com)... 74.112.186.157
Connecting to keio.app.box.com (keio.app.box.com)|74.112.186.157|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://public.boxcloud.com/d/1/b1!yLFCz

In [2]:
  obj_labels = (
    'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
    'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
    'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle',
    'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
    'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
    'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven',
    'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush')

必要なライブラリを読み込む

In [3]:
from torch.nn.modules.batchnorm import BatchNorm2d
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
import torchvision.transforms as T
import torchvision.transforms.functional as FF
from torchvision.ops.boxes import box_iou
from torchvision.io import read_image
from torchvision.utils import draw_bounding_boxes
import glob
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from six import b
import cv2
from PIL import Image
import math
import matplotlib.pyplot as plt

各種設定

In [4]:
image_path = "coco128/images/train2017"
label_path = "coco128/labels/train2017"
img_list = sorted(glob.glob(os.path.join(image_path,"*")))
label_list = sorted(glob.glob(os.path.join(label_path,"*")))
img_size = 416
class_n = 80

## データの前処理

YoLov3で用いるデータセットを準備する
- 様々な工夫が施されている
- 前処理の説明であるが、YoLov3の本質が詰まっている

### AnchorBoxの定義
YoLoは通常AnchorBoxを利用するため、これを設定する
- アンカーボックスは、高さと幅が事前に決められた境界ボックスのセットのことで、一般にメッシュを複数個組み合わせた矩形となる
  - 検出する特定の物体のスケールおよび縦横比を取得するため利用し、学習データセットに含まれるオブジェクトサイズに基づいて選択、事前作成されるのが一般的である
- ここではk-Meansを利用してデータセットのアノテーションをクラスタリングし、その結果をAnchorBoxとする
  - 3つのPrior Boxの入手にこのAncorBoxが関わっている
  - K-means clusteringを利用してCOCOデータセットにある全てのBounding Boxを9個に分類する
    - スケールが3つ、BOXが3つであるため9個

データからクラスタリングを行い、各データがどのクラスに属するか決定しておく
- YoLov3はAnchorBoxが9個あるため、9つのクラスタリングを行う

In [5]:
bbox_dict = {'width':[] , 'height':[]}
bbox_list = []
for path in label_list:
  with open(path , 'r',newline='\n') as f:
      for s_line in f: # Bounding Boxが順に記述されたファイル
        bbox = [float(x) for x in s_line.rstrip('\n').split(' ')] # 改行で行を分けて各Bounding Boxの行を取得、さらにスペースで分割する
        bbox_dict['width'].append(bbox[3])
        bbox_dict['height'].append(bbox[4])
        bbox_list.append(bbox[3:5])
df = pd.DataFrame(bbox_dict)
print(df.head()) # 先頭5このファイルについてwidthとhightの抽出結果を表示
km = KMeans(n_clusters=9,
  init='random',
  n_init=10,
  max_iter=300,
  tol=1e-04,
  random_state=0)
y_km = km.fit_predict(bbox_list)
df['cluster'] = y_km
print(df.head()) # 先頭5このファイルについてどこのクラスタに配属されたかを表示

      width    height
0  0.955609  0.595500
1  0.498875  0.476417
2  0.494125  0.510583
3  0.678875  0.781500
4  0.118047  0.096937
      width    height  cluster
0  0.955609  0.595500        4
1  0.498875  0.476417        6
2  0.494125  0.510583        6
3  0.678875  0.781500        0
4  0.118047  0.096937        2


9個のAnchorBoxを決定
- 大きいAnchorBox(先頭行)は大きな物体の検出に用いるため、大きな物体を検出する出力(Scale 3)に割り当てる
- 逆に小さいAnchorBoxは小さな物体の検出に用いるため、小さな物体を検出する出力(Scale 1)に割り当てる

In [6]:
anchor_dict = {"width":[],"height":[],"area":[]}
for i in range(9):
  anchor_dict["width"].append(df[df["cluster"] == i].mean()["width"])
  anchor_dict["height"].append(df[df["cluster"] == i].mean()["height"])
  anchor_dict["area"].append(df[df["cluster"] == i].mean()["width"]*df[df["cluster"] == i].mean()["height"])
anchor = pd.DataFrame(anchor_dict).sort_values('area', ascending=False)
anchor["type"] = [int(img_size/32) ,int(img_size/32) ,int(img_size/32) ,  int(img_size/16) ,int(img_size/16) ,int(img_size/16) , int(img_size/8), int(img_size/8), int(img_size/8)]
print(anchor)

      width    height      area  type
4  0.953643  0.856002  0.816320    13
0  0.591571  0.828589  0.490169    13
5  0.777525  0.379511  0.295079    13
6  0.414800  0.443691  0.184043    26
7  0.245450  0.694372  0.170434    26
8  0.152484  0.359089  0.054755    26
3  0.245576  0.165619  0.040672    52
2  0.087694  0.152691  0.013390    52
1  0.040143  0.047590  0.001910    52


### グリッド、アンカーボックス、Bouding Boxの関係

YoLov3におけるグリッドとAnchor Box、Bounding Boxは、次の図のように定義される

ここで
- $C_x, C_y$は1ピクセルの幅、
- $t_x , t_y , t_w , t_h$はYOLOv3の予測
- $P_w , P_h$はAnchorBoxサイズ

をそれぞれ表す

<img src="http://class.west.sd.keio.ac.jp/dataai/text/yologrid.png" width=300>


### 幅と高さの推定

YoLoV3では次の式で幅と高さを推定する
- $b_w = P_w e^{t_w}$
- $b_h = P_h e^{t_h}$

YOLOv3の出力にある$t_w,_h$の表現として、小さいときは正確に、大きい場合は例えば1ドットのずれなどは殆ど気にならないため、指数関数を用いてAnchorBoxを表現する
- 直接物体の幅や高さをピクセル値予測せず、AnchorBoxを調整するという表現にすることで、学習コストを削減できる

### 中心位置の推定方法
YoLov3では次の式で中心位置を推定する
- $b_x = \sigma(t_x) + C_x$
- $b_y = \sigma(t_y) + C_y$

直接的に推定してもよいが、なぜか複雑な式で予測している

- 例えばサイズが$416 \times 416$の画像に物体が写っており、その物体の中心が$(200 ,250)$であったとする

<img src="http://class.west.sd.keio.ac.jp/dataai/text/yoloapple.png" width=300>

- 対称物体が大きい場合、小さい特徴マップで予測することになる
  - $13 \times 13$の特徴マップでは画像サイズに対する中心の位置は$\frac{200}{416} = 0.481, \frac{250}{416} = 0.600$となるため、$13 \times 13$pxでは最も近いのは(6.253 , 7.800)となり、結果として小数点を省いた$13\times 13$メッシュにおける座標値としての(6, 7)が中央となる

このままでは、特徴マップが小さくなるにつれて誤差が大きくなる
- これを防ぐため、YoLov3ではピクセル内のどこに中心があるかを予測する
- YoLov3の出力$t_x , t_y$をシグモイド関数に入力して0から1の値に変換
  - メッシュよりも細かい場所の指定(メッシュに対して何割のところに中心があるか)を知ることができる
- $(6.253 , 7.800)$は、対象とするメッシュでピクセル(7 , 6)であり、そこから(0.253 , 0.800)の場所に中心があると考えることができる
- これを受けて、$0.253 = \sigma(t_x)$となるような$x$を推論する

<img src="http://class.west.sd.keio.ac.jp/dataai/text/yoloapplegrid.png" width=300>


### YoLov3の本質

YoLov3の予測はAnchorBoxの微調整やピクセル間の誤差補正予測など、微調整をふんだんに行っていることがわかる
- そのために、少々値が加工された入力を必要とする

アノテーションデータは真の中心位置と幅・高さを保持しているため、これをYoLov3用に変換する必要がある

- 幅と高さについて、
  - $b_w = P_w e^{t_w}$
  - $b_h = P_h e^{t_h}$
  
  であるため、

  $t_w , t_h$は
  - $t_w = \ln\frac{b_w}{P_w}$
  - $t_h = \ln\frac{b_h}{P_h}$

  となる
  - この処理は`def wh2twth`で記述されている

- 中心位置についても、
  - $b_x = \sigma(t_x) + C_x$
  - $b_y = \sigma(t_y) + C_y$

  であるため、
  - $b_x - C_x = \sigma(t_x)$
  - $b_y - C_y = \sigma(t_y)$

  となり、この逆関数を求めると$t_x$と$t_y$を得ることができる

  シグモイド関数の逆関数はロジット関数であるため、
  - $t_x = \ln\frac{b_x - C_x}{1-(b_x - C_x)}$
  - $t_y = \ln\frac{b_y - C_y}{1-(b_y- C_y)}$

  となる
  - この処理は`def cxcy2txty`で記述されている

以上を踏まえて実際にデータセットの前処理部分を記述する
- 基本的にはDataset関数の準備であり、その核心は、`__getitem__`によりデータを取得することと、`__len__`により全データ数を返すところ
- その他はすべて`__getitem__`のための副次的な処理関数

In [7]:
class YOLOv3_Dataset(Dataset):
  def __init__(self, img_list, label_list, class_n, img_size, anchor_dict, transform):
    super().__init__()
    self.img_list = img_list
    self.label_list = label_list
    self.anchor_dict = anchor_dict
    self.class_n = class_n
    self.img_size = img_size
    self.transform = transform
    self.anchor_iou = torch.cat([torch.zeros(9,2) , torch.tensor(self.anchor_dict[["width","height"]].values)] ,dim = 1)
    self.map_size = [int(self.img_size/32) , int(self.img_size/16) , int(self.img_size/8)]

  def get_label(self, path):  # coco128/labelsにあるラベルとBBoxがセットになったファイルを読み出す
    bbox_list = []
    with open(path , 'r',newline='\n') as f:
      for s_line in f:
        bbox = [float(x) for x in s_line.rstrip('\n').split(' ')]
        bbox_list.append(bbox)
    return bbox_list

  def wh2twth(self, wh):
    twth = []
    for i in range(9):
      anchor = self.anchor_dict.iloc[i]
      aw = anchor["width"]
      ah = anchor["height"]
      twth.append([math.log(wh[0]/aw) , math.log(wh[1]/ah)])
    return twth

  def cxcy2txty(self,cxcy):
    txty = []
    for size in self.map_size:
      grid_x = int(cxcy[0]*size)
      grid_y = int(cxcy[1]*size)
      tx = math.log((cxcy[0]*size - grid_x + 1e-10) / (1 - cxcy[0]*size +grid_x+ 1e-10))
      ty = math.log((cxcy[1]*size - grid_y+ 1e-10) / (1 - cxcy[1]*size + grid_y+ 1e-10))
      txty.append([grid_x , tx , grid_y ,ty])
    return txty

  def label2tensor(self, bbox_list):
    tensor_list = []
    for size in self.map_size:
      for x in range(3):
        tensor_list.append(torch.zeros((4 + 1 + self.class_n,size,size)))
    for bbox in bbox_list:
      cls_n = int(bbox[0])
      txty_list = self.cxcy2txty(bbox[1:3])
      twth_list = self.wh2twth(bbox[3:])
      label_iou = torch.cat([torch.zeros((1,2)), torch.tensor(bbox[3:]).unsqueeze(0)],dim=1)
      iou = box_iou(label_iou, self.anchor_iou)[0]
      obj_idx = torch.argmax(iou).item()
#      print(cls_n, txty_list, twth_list, label_iou, iou, obj_idx)
      for i, twth in enumerate(twth_list):
        tensor = tensor_list[i]
        txty = txty_list[int(i/3)]
        if i == obj_idx:
          tensor[0, txty[2], txty[0]] = txty[1]
          tensor[1, txty[2], txty[0]] = txty[3]
          tensor[2, txty[2], txty[0]] = twth[0]
          tensor[3, txty[2], txty[0]] = twth[1]
          tensor[4, txty[2], txty[0]] = 1
          tensor[5+cls_n, txty[2], txty[0]] = 1
    scale3_label = torch.cat(tensor_list[0:3] , dim = 0)
    scale2_label = torch.cat(tensor_list[3:6] , dim = 0)
    scale1_label = torch.cat(tensor_list[6:] , dim = 0)
    return scale3_label , scale2_label , scale1_label

  def __getitem__(self, idx):
    img_path = self.img_list[idx]
    label_path = self.label_list[idx]
    bbox_list = self.get_label(label_path)
    img = cv2.imread(img_path)
    img = cv2.resize(img , (self.img_size , self.img_size))
    img = Image.fromarray(img)
    img = self.transform(img)
    scale3_label , scale2_label , scale1_label = self.label2tensor(bbox_list)
    return img , scale3_label , scale2_label , scale1_label

  def __len__(self):
    return len(self.img_list)

学習のためにBouding Boxや中心位置などを操作しているため、元に戻す関数`visualization`を定義する

In [8]:
def visualization(y_pred, anchor, img_size,conf = 0.5, is_label = False):
  size = y_pred.shape[2]
  anchor_size = anchor[anchor["type"] == size]
  bbox_list = []
  obj_list = []
  for i in range(3):
    a = anchor_size.iloc[i]
    grid = img_size/size
    y_pred_cut = y_pred[0,i*(4 + 1 + class_n) :(i+1)*(4 + 1 + class_n) ].cpu()
    if is_label:
      y_pred_conf = y_pred_cut[4,:,:].numpy()
    else:
      y_pred_conf = torch.sigmoid(y_pred_cut[4,:,:]).numpy()
    index = np.where(y_pred_conf > conf)
    for y,x in zip(index[0],index[1]):
      cx = x*grid + torch.sigmoid(y_pred_cut[0,y,x]).numpy()*grid
      cy = y*grid + torch.sigmoid(y_pred_cut[1,y,x]).numpy()*grid
      width = a["width"]*torch.exp(y_pred_cut[2,y,x]).numpy()*img_size
      height = a["height"]*torch.exp(y_pred_cut[3,y,x]).numpy()*img_size
      xmin,ymin,xmax,ymax = cx - width/2 , cy - height/2 ,cx + width/2 , cy + height/2
      bbox_list.append([xmin,ymin,xmax,ymax])
      obj_list.append(y_pred_cut[5:,y,x].argmax()) # 正解ラベル
  return bbox_list, obj_list

画像を表示するための関数`show`を定義する

In [9]:
def show(imgs):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fix, axs = plt.subplots(ncols=len(imgs), squeeze=False, dpi=150)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = FF.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

試しに、学習データを表示させる
- Bounding Boxの色は、Scaleの違いを表している
  - 赤：大サイズとして予測（Scale3）
  - 緑：中サイズとして予測（Scale2）
  - 青：小サイズとして予測（Scale1）

In [10]:
transform = T.Compose([T.ToTensor()])
train_data = YOLOv3_Dataset(img_list, label_list, 80, img_size , anchor, transform)
train_loader = DataLoader(train_data, batch_size = 1)

for n, (img, scale3_label, scale2_label, scale1_label) in enumerate(train_loader):
  path = img_list[n]
  img = cv2.imread(path)[:,:,::-1]
  img = cv2.resize(img, (img_size, img_size))
  img = torch.tensor(img.transpose(2,0,1))
  preds = [scale3_label, scale2_label, scale1_label]
  for color,pred in zip(["red","green","blue"],preds):
    bbox_list, obj_list = visualization(pred, anchor, img_size, conf=0.9, is_label=True)
    if len(bbox_list) != 0:
      obj_label_list = []
      for obj_item in obj_list:
        obj_label_list.append(obj_labels[obj_item])
      img = draw_bounding_boxes(img, torch.tensor(bbox_list), colors=color, width=1, labels=obj_label_list)
  show(img)
  if n == 10:
    break
print(scale3_label.shape, scale2_label.shape, scale1_label.shape)

Output hidden; open in https://colab.research.google.com to view.

## YoLov3ネットワーク


ちょっと厄介ではあるが、図の通りに実装する
- n=80つまり、80クラス分類であるが、nであるため自由に変更できるようにしている

In [11]:
class YOLOv3(nn.Module):
  def __init__(self,class_n = 80):
    super(YOLOv3 , self).__init__()
    self.class_n = class_n
    self.first_block = nn.Sequential(
        nn.Conv2d(3 , 32 , 3 , 1 , 1),
        nn.BatchNorm2d(32),
        nn.LeakyReLU(),
        nn.Conv2d(32 , 64 , 3 , 2 , 1),
        nn.BatchNorm2d(64),
        nn.LeakyReLU(),
      )
    self.residual_block_1 = self.MakeResidualBlock(64)
    self.conv_1 = nn.Conv2d(64 , 128  , 3 , 2 , 1)
    self.residual_block_2 = nn.Sequential(self.MakeResidualBlock(128),self.MakeResidualBlock(128))
    self.conv_2 = nn.Conv2d(128 , 256  , 3 , 2 , 1)
    self.residual_block_3 = nn.Sequential(self.MakeResidualBlock(256),self.MakeResidualBlock(256),self.MakeResidualBlock(256),self.MakeResidualBlock(256),self.MakeResidualBlock(256),self.MakeResidualBlock(256),self.MakeResidualBlock(256),self.MakeResidualBlock(256))
    self.conv_3 = nn.Conv2d(256 , 512  , 3 , 2 , 1)
    self.residual_block_4 = nn.Sequential(self.MakeResidualBlock(512),self.MakeResidualBlock(512),self.MakeResidualBlock(512),self.MakeResidualBlock(512),self.MakeResidualBlock(512),self.MakeResidualBlock(512),self.MakeResidualBlock(512),self.MakeResidualBlock(512))
    self.conv_4 = nn.Conv2d(512 , 1024  , 3 , 2 , 1)
    self.residual_block_5 = nn.Sequential(self.MakeResidualBlock(1024),self.MakeResidualBlock(1024),self.MakeResidualBlock(1024),self.MakeResidualBlock(1024),)
    self.conv_block = nn.Sequential(self.MakeResidualBlock(1024),self.MakeResidualBlock(1024),self.MakeResidualBlock(1024),self.MakeResidualBlock(1024),)
    self.scale3_output = nn.Conv2d(1024 , (3 * (4 + 1 +self.class_n)) , 1 , 1 )
    self.scale2_upsample = nn.Conv2d(1024 , 256 , 1 , 1 )
    self.scale2_convblock = nn.Sequential(nn.Sequential(nn.Conv2d(768 , 256 , 1 , 1),
                          nn.BatchNorm2d(256),
                          nn.LeakyReLU(),
                          nn.Conv2d(256 , 512 , 3 , 1 ,  1),
                          nn.BatchNorm2d(512),
                          nn.LeakyReLU(),
                          ),self.MakeResidualBlock(512),self.MakeResidualBlock(512),)
    self.scale2_output = nn.Conv2d(512 , (3 * (4 + 1 +self.class_n)) , 1 , 1 )
    self.scale1_upsample = nn.Conv2d(512 , 128 , 1 , 1 )
    self.scale1_convblock = nn.Sequential(nn.Sequential(nn.Conv2d(384 , 128 , 1 , 1),
                          nn.BatchNorm2d(128),
                          nn.LeakyReLU(),
                          nn.Conv2d(128 , 256 , 3 , 1 ,  1),
                          nn.BatchNorm2d(256),
                          nn.LeakyReLU(),
                          ),self.MakeResidualBlock(256),self.MakeResidualBlock(256),)
    self.scale1_output =  nn.Conv2d(256 , (3 * (4 + 1 +self.class_n)) , 1 , 1 )
    self.upsample = nn.Upsample(scale_factor = 2)
  def MakeResidualBlock(self,fn):
    block = nn.Sequential(nn.Conv2d(fn , int(fn/2) , 1 , 1),
                          nn.BatchNorm2d(int(fn/2)),
                          nn.LeakyReLU(),
                          nn.Conv2d(int(fn/2) , fn , 3 , 1 ,  1),
                          nn.BatchNorm2d(fn),
                          nn.LeakyReLU(),
                          )
    return block
  def forward(self,x):
    x = self.first_block(x)
    x_res = self.residual_block_1(x)
    x = x + x_res
    x = self.conv_1(x)
    for layer in self.residual_block_2:
      x_res = layer(x)
      x = x + x_res
    x = self.conv_2(x)
    for layer in self.residual_block_3:
      x_res = layer(x)
      x = x + x_res
    x1 = x
    x = self.conv_3(x)
    for layer in self.residual_block_4:
      x_res = layer(x)
      x = x + x_res
    x2 = x
    x = self.conv_4(x)
    for layer in self.residual_block_5:
      x_res = layer(x)
      x = x + x_res
    for layer in self.conv_block:
      x = layer(x)
    scale3_result = self.scale3_output(x)
    scale2_up = self.upsample(self.scale2_upsample(x))
    x = torch.cat([x2 , scale2_up],dim = 1)
    for layer in self.scale2_convblock :
      x = layer(x)
    x2 = x
    scale2_result = self.scale2_output(x)
    scale1_up = self.upsample(self.scale1_upsample(x2))
    x = torch.cat([x1 , scale1_up],dim = 1)
    for layer in self.scale1_convblock :
      x = layer(x)
    scale1_result = self.scale1_output(x)
    return  scale3_result , scale2_result , scale1_result
model =YOLOv3().cuda()

In [12]:
with torch.no_grad():
  output = model(torch.zeros((1,3,416,416)).cuda())
for i in range(3):
  print(output[i].shape)

torch.Size([1, 255, 13, 13])
torch.Size([1, 255, 26, 26])
torch.Size([1, 255, 52, 52])


modelとしてインスタンス化
- ここで、実際に出力を確認してみる
- $13 \times 13$、$26 \times 26$、$52 \times 52$のグリッドそれぞれについて、255要素のデータが出力されており、設計通りであることがわかる

## ロス関数

YoLov3のロス関数はBounding Boxをラベルの両方を考慮しており、かなり複雑

論文では次の式が与えられている
- 説明のため、順番を並び替えている

$$\lambda_{coord} \sum^{S^2}_{i=0} \sum^B_{j=0}\mathfrak{1}^{obj}_{i,j}[(t_x−\hat{t}_x)^2+(t_y−\hat{t}_y)^2+(t_w−\hat{t}_w)^2+(_th−\hat{t}_h)^2]\\ +\sum^{S^2}_{i=0} \sum^B_{j=0}\mathfrak{1}^{obj}_{i,j}[-log(\sigma(t_o))+\lambda_{noobj}\sum^{S^2}_{i=0} \sum^B_{j=0}\mathfrak{1}^{noobj}_{i,j}[-log(1-\sigma(t_o))]\\+\sum^C_{k=1}BCE(\hat{y}_k,\sigma(s_k))] $$

簡単に説明する
- 1行目はBounding Boxの$x, y, w, h$それぞれの誤差を表しており、単純に二乗誤差が用いられている
- 2行目はobjective scoreに沿うとして、オブジェクトがセルの中に存在するかどうかで2つの項に分かれている
  - 存在することを正しく判定できたか、また、存在しないことを正しく判定できたかをそれぞれ表している
  - 検出対象の物体がない矩形は objectness score が0になるように学習する必要があり、これに対応する
- 3行目はクラスつまり物体のラベルを正しく推定できたかを意味しており、BCEロスを利用している
  - なお、コード上では`BCEWithLogitsLoss`を利用している
  - `BCELoss`はシグモイドをとった後の値を入力として受け付け、`BCEWithLogitsLoss`はシグモイドをとる前の値を入力として受け付けつける点に注意

なお、論文では、$t_x, t_y$の損失は二乗誤差で計算すると書いてあり、ここではその通りに実装している
- 論文で示されているdarknet実装では、シグモイド関数適用後の$\sigma(t_x), \sigma(t_y)$の値との差分を求めているため、論文とは異なっている
- この実装に従う場合は、$t_x, t_y$はバイナリクロスエントロピーで損失を計算することになる

In [13]:
optimizer = torch.optim.Adam(model.parameters())
criterion_bce = torch.nn.BCEWithLogitsLoss()
def bbox_metric(y_pred , y_true,class_n = 80):
  for i in range(3):
    y_pred_cut = y_pred[:,i*(4 + 1 + class_n) :(i+1)*(4 + 1 + class_n) ]
    y_true_cut = y_true[:,i*(4 + 1 + class_n) :(i+1)*(4 + 1 + class_n) ]
    loss_coord = torch.sum(torch.square(y_pred_cut[:,0:4] - y_true_cut[:,0:4])*y_true_cut[:,4])
    loss_obj = torch.sum((-1 * torch.log(torch.sigmoid(y_pred_cut[:,4] )+ 1e-10) + criterion_bce(y_pred_cut[:,5:],y_true_cut[:,5:]))*y_true_cut[:,4])
    loss_noobj =  torch.sum((-1 * torch.log(1 - torch.sigmoid(y_pred_cut[:,4])+ 1e-10))*(1 - y_true_cut[:,4]))
    return loss_coord , loss_obj , loss_noobj

## 学習
最新GPU(RTX3090Ti)でも1時間弱かかる
- Google Colaboratoryでは、ほぼ実行時間限界の半分(10時間)程度必要となる可能性がある


In [14]:
from tqdm import tqdm
lambda_coord = 1
lambda_obj = 10
lambda_noobj = 1
conf = 0.5
best_loss = 99999
for epoch in range(300):
  total_train_loss = 0
  total_train_loss_coord = 0
  total_train_loss_obj = 0
  total_train_loss_noobj = 0
  with tqdm(train_loader) as pbar:
    pbar.set_description("[train] Epoch %d" % epoch)
    for n , (img , scale3_label , scale2_label ,scale1_label) in enumerate(pbar):
      optimizer.zero_grad()
      img = img.cuda()
      scale1_label = scale1_label.cuda()
      scale2_label = scale2_label.cuda()
      scale3_label = scale3_label.cuda()
      labels = [scale3_label , scale2_label ,scale1_label]
      preds  = list(model(img))
      loss_coord = 0
      loss_obj = 0
      loss_noobj = 0
      for label , pred in zip(labels , preds):
        _loss_coord , _loss_obj , _loss_noobj = bbox_metric(pred , label)
        loss_coord += _loss_coord
        loss_obj += _loss_obj
        loss_noobj += _loss_noobj
      loss = lambda_coord*loss_coord + lambda_obj*loss_obj + lambda_noobj*loss_noobj
      total_train_loss += loss.item()
      total_train_loss_coord += loss_coord.item()
      total_train_loss_obj += loss_obj.item()
      total_train_loss_noobj += loss_noobj.item()
      loss.backward()
      optimizer.step()
      pbar.set_description("[train] Epoch %d loss %f loss_coord %f loss_obj %f loss_noobj %f" % (epoch ,total_train_loss/(n+1),total_train_loss_coord/(n+1) , total_train_loss_obj/(n+1),total_train_loss_noobj/(n+1)))
      if best_loss > total_train_loss/(n+1):
        model_path = 'model.pth'
        torch.save(model.state_dict(), model_path)
        best_loss = total_train_loss/(n+1)

[train] Epoch 0 loss 267.460275 loss_coord 11.837668 loss_obj 5.359628 loss_noobj 202.026326: 100%|██████████| 128/128 [03:48<00:00,  1.79s/it]
[train] Epoch 1 loss 97.874924 loss_coord 8.557057 loss_obj 6.628440 loss_noobj 23.033469: 100%|██████████| 128/128 [00:24<00:00,  5.20it/s]
[train] Epoch 2 loss 98.076755 loss_coord 10.716055 loss_obj 7.037160 loss_noobj 16.989101: 100%|██████████| 128/128 [00:12<00:00,  9.94it/s]
[train] Epoch 3 loss 93.803527 loss_coord 9.039503 loss_obj 6.916220 loss_noobj 15.601822: 100%|██████████| 128/128 [00:13<00:00,  9.61it/s]
[train] Epoch 4 loss 90.849114 loss_coord 7.300317 loss_obj 6.901610 loss_noobj 14.532693: 100%|██████████| 128/128 [00:13<00:00,  9.76it/s]
[train] Epoch 5 loss 87.410906 loss_coord 7.014522 loss_obj 6.640231 loss_noobj 13.994070: 100%|██████████| 128/128 [00:12<00:00, 10.23it/s]
[train] Epoch 6 loss 87.268542 loss_coord 6.833284 loss_obj 6.648030 loss_noobj 13.954953: 100%|██████████| 128/128 [00:12<00:00, 10.27it/s]
[train] E

In [15]:
lambda_coord = 1
lambda_obj = 10
lambda_noobj = 1

## 結果確認

まぁまぁといったところか
- 学習量も少なく、データ拡張も行っていないため、この程度であろう

In [16]:
model_path = 'model.pth'
model.load_state_dict(torch.load(model_path))
model = model.cuda()
for path in img_list:
  img = cv2.imread(path)
  img = cv2.resize(img , (img_size , img_size))
  img = Image.fromarray(img)

  img = transform(img).unsqueeze(0).cuda()
  with torch.no_grad():
    preds  = list(model(img))
  img = cv2.imread(path)[:,:,::-1]
  img = cv2.resize(img , (img_size , img_size))
  img = torch.tensor(img.transpose(2,0,1))
  for color,pred in zip(["red","green","blue"],preds):
    bbox_list, obj_list = visualization(pred, anchor, img_size, conf = 0.9)
    if len(bbox_list) != 0:
      obj_label_list = []
      for obj_item in obj_list:
        obj_label_list.append(obj_labels[obj_item])
      img = draw_bounding_boxes(img, torch.tensor(bbox_list), colors=color, width=1, labels=obj_label_list)
  show(img)

Output hidden; open in https://colab.research.google.com to view.

# YoLoのその後

YoLoの生みの親、Joseph Redmonは、先に述べた通りYoLo V3で開発をやめ、ソーシャルメディアも閉じている

YoLov3の論文も、それまでとはことなり、口語調で記述されており、ある意味雑な論文である
- 例えば、次のような文章から論文は始まる"We present some updates to YOLO! We made a bunch of little design changes to make it better. We also trained this new network that's pretty swell. It's a little bigger than last time but more accurate. It's still fast though, don't worry."
- 訳(DeepL)：YOLO！のいくつかのアップデートを紹介します。より良いものにするために、たくさんの小さなデザイン変更を行いました。また、この新しいネットワークはとても素晴らしいものです。前回より少し大きくなりましたが、より正確になっています。それでもまだ速いので、ご心配なく。

開発から退いた理由も、論文の最後に記載されている

- "What are we going to do with these detectors now that we have them?"
- 訳：このような物体検出器を使って何をするんだい？

- "A lot of the people doing this research are at Google and Facebook. I guess at least we know the technology is in good hands and deﬁnitely won't be used to harvest your personal information and sell it to…. wait, you're saying that's exactly what it will be used for?? Oh."
- 訳(DeepL)：この研究をしている人たちの多くは、GoogleやFacebookにいます。少なくとも私たちは、この技術が良い人たちの手に渡り、あなたの個人情報を採取して売るために使われることはないだろうと思っています。ああ。

- "Well the other people heavily funding vision research are the military and they've never done anything horrible like killing lots of people with new technology oh wait….."
- 訳(DeepL)：ビジョン研究に多額の資金を提供しているのは軍で、彼らは新しい技術で多くの人を殺すような恐ろしいことはしていません。おっとまてよ...

- "The author is funded by the Ofﬁce of Naval Research and Google."
- 訳(DeepL)：著者は、Ofﬁce of Naval ResearchとGoogleから資金提供を受けている。

これらの記述から、研究成果の悪用が許せなかったと考えられます
- YoLoを生み出したことで、その責任が問われるような事態を想定したのかもしれません


最後に、次のようなtwitterを投稿している

<img src="http://class.west.sd.keio.ac.jp/dataai/text/jrtwitter.png" width=500>


# YOLOのさらにその後


YoLoという名前は物体検出において著名なため、その後もYoLoという名前で開発は継続されている

## YOLOv4

- YOLOv4はシングルショットで物体を検出するYOLOシリーズであり、YOLOv3までの作者のJoseph Redmonは2020年2月に開発終了を発表した後、YOLOv4はDarknetのWindows版を開発しているAlexey Bochkovskyによって開発された

特長として、YOLOv4はBackboneにCSPDarknet53、NeckにSPP(Spatial pyramid pooling)、PAN(Path Aggregation Network)、HeadにYOLOv3を使用している

- CPSDarknetの基本であるCSPNetはFeatureMapの一部のみをConvして残りをConcatすることで精度を落とさずに高速化する手法

- SPPは複数のカーネルサイズ（1,5,9,13）で同時にPoolingすることで精細な情報と広域な情報の両方を取得する方法

- PANは異なるバックボーンレベルから特徴量をDetectorに伝えることで、入力に近い層の情報を活用する手法

YOLOv3と比較して、mAP(Mean Average Precision)が向上している

## YOLOv5

Ultralytics社により開発されたモデルであり、YOLOv4よりも性能が向上したとされている

商用利用が限定されている

## YOLOv6

2023年1月に更新されYOLOv6(v3.0)となった
- YOLOv7やYOLOv8の性能を上回っている

YOLO開発者間で競うという謎な状態

## YOLOv7

YOLOv7は、YOLOv4、Scaled-YOLOv4, YOLORと同じグループが開発した

特長
- 基本アーキテクチャにELAN, E-ELANを使用  
DenseNetやCSPDarknetなどのconcatenateベースのモデルを進化させた構造
- 複合スケーリング  
concatenateモデルに適した複合スケーリング方法であり、モデルをスケーリングする際にはその一部だけではなく、モデルのdepth(層数)とwidth(ノード数)などの、全体を同時にスケーリングさせる手法である
- re-parameterizationの適用  
RepVGGなどと同じ手法を適用し、学習時は推論時と別の構造を持つネットワークを用いることで、ラベル割り当て戦略の最適化している
- ベースのラベル割り当て戦略にOTAを利用
- Auxiliary Lossを併用してラベル割り当てを行う

## YOLOv8

YOLOv8は、YOLOv5の公開元であるUltralytics社が公開
- 大規模データセットでの学習だけでなく、object detection、segmentation、classificationタスクで利用可能である
- モデルや損失関数が更新されており、nchor-free detection headが導入されている

# 課題
データオーグメンテーションを実装しなさい
- どのような形でも構わないが、ここではBounding Boxも変換することを忘れずに